In [31]:
import pandas as pd
import numpy as np

from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

import os
import string
import re

from numpy import array
from numpy import asarray
from numpy import zeros

from sklearn.model_selection import train_test_split

In [28]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [11]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [64]:
id='1Pk5MK9Hs_kMUT9NotGnOKE0NPra-39YU'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('train.csv')

train = pd.read_csv('train.csv')

id='1GsTM9oLtIV8-Da_fDOFWsQYMpgQ8GOYJ'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('test.csv')

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [4]:
def remove_punct(text):
    #filtrado de signos de puntuacion
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    return text

In [65]:
y = train.target
train['text'] = train['text'].apply(lambda x: remove_punct(x))
x_corpus, x_validation, y_corpus, y_validation = train_test_split(train.text, y, test_size=0.3, stratify=y)

In [66]:
corpus = list(x_corpus)

In [67]:
len(corpus)

5329

In [68]:
corpus

['meek mill begging nicki minaj to let him obliterate ovofest nowplaying httptcoxomi4zqzgp httptco0m1tw3datd',
 'legionnaires disease whats being done to stop deadly outbreak httptcofftokd0vts ',
 'kanger coils  burning out fast via rvaping101 httptcocykr4xaluh',
 'detonation into the realistic assets entering india koz httptco9zrqmd8ngz',
 'ronda rousey would be close to making floyd mayweathers money in 50 fights  bloody elbow httptcoijzcytbffo boxing',
 'you cant fight fate and you cant survive alone i cant help but notice that almost seems like a definition of who i am',
 'turnedonfetaboo hsjb215 check out this rockin preview of claytonbryant danger zone coming soon httpstcoe1wrvyzfkv artistsunited',
 'as a cycling fan i feel sorry for world athletics doping is a blight exacerbated monetary reward a lot of soul searching will be required',
 'police assailant in latest us movie theatre attack was homeless had psychological issues httptcozdcvlyq6qk',
 'tanslash waiting for an ambulan

In [69]:
#DEL TRAIN SPLITEADO

#buscamos vocabulary (palabras unicas)
word_tokenizer_train = Tokenizer()
word_tokenizer_train.fit_on_texts(corpus)

#cantidad de palabras unicas
vocab_length = len(word_tokenizer_train.word_index) + 1

#convertimos los tweets a numeros
embedded_sentences_train = word_tokenizer_train.texts_to_sequences(corpus)

#completamos con ceros en las oraciones mas cortas para dejar todos los vectores con la misma dimension (la dimension de la oracion mas larga)
from nltk.tokenize import word_tokenize

word_count_train = lambda sentence: len(word_tokenize(sentence))
longest_sentence_train = max(corpus, key=word_count_train)
length_long_sentence_train = len(word_tokenize(longest_sentence_train))
padded_sentences_train = pad_sequences(embedded_sentences_train, length_long_sentence_train, padding='post')

In [70]:
len(padded_sentences_train)

5329

In [71]:
#convertimos los tweets DE VALIDACION a numeros
word_tokenizer_val = Tokenizer()
word_tokenizer_val.fit_on_texts(list(x_validation))
embedded_sentences_val = word_tokenizer_val.texts_to_sequences(list(x_validation))

#completamos con ceros en las oraciones mas cortas para dejar todos los vectores con la misma dimension (la dimension de la oracion mas larga)
word_count_val = lambda sentence: len(word_tokenize(sentence))
longest_sentence_val = max(corpus, key=word_count_val)
length_long_sentence_val = len(word_tokenize(longest_sentence_val))
padded_sentences_val = pad_sequences(embedded_sentences_val, length_long_sentence_val, padding='post')

In [72]:
#convertimos los tweets DEL TEST a numeros
word_tokenizer_test = Tokenizer()
word_tokenizer_test.fit_on_texts(list(test.text))
embedded_sentences_test = word_tokenizer_test.texts_to_sequences(list(test.text))

#completamos con ceros en las oraciones mas cortas para dejar todos los vectores con la misma dimension (la dimension de la oracion mas larga)
word_count_test = lambda sentence: len(word_tokenize(sentence))
longest_sentence_test = max(corpus, key=word_count_test)
length_long_sentence_test = len(word_tokenize(longest_sentence_test))
padded_sentences_test = pad_sequences(embedded_sentences_test, length_long_sentence_test, padding='post')

In [73]:
#convertimos los tweets DEL TRAIN COMPLETO a numeros
word_tokenizer = Tokenizer()
word_tokenizer.fit_on_texts(list(train.text))
embedded_sentences = word_tokenizer.texts_to_sequences(list(train.text))

#cantidad de palabras unicas
vocab_length = len(word_tokenizer.word_index) + 1

#completamos con ceros en las oraciones mas cortas para dejar todos los vectores con la misma dimension (la dimension de la oracion mas larga)
word_count = lambda sentence: len(word_tokenize(sentence))
longest_sentence = max(corpus, key=word_count)
length_long_sentence = len(word_tokenize(longest_sentence))
padded_sentences = pad_sequences(embedded_sentences, length_long_sentence, padding='post')

In [74]:
#vinculamos nuestros valores a los de GloVe
import json

embeddings_dictionary = {}
f = open('glove.6B.100d.txt', encoding="utf8")
for line in f:
  values = line.split()
  word = values[0]
  vector_dimension = np.asarray(values[1:])
  embeddings_dictionary[word] = vector_dimension
f.close()

In [75]:
embedding_matrix = zeros((vocab_length, 100))
for word, index in word_tokenizer_train.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

DEEP LEARNING 1

In [76]:
"""model = Sequential()
embedding_layer = Embedding(vocab_length, 100, weights=[embedding_matrix], input_length=length_long_sentence, trainable=False)
model.add(embedding_layer)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))"""

"model = Sequential()\nembedding_layer = Embedding(vocab_length, 100, weights=[embedding_matrix], input_length=length_long_sentence, trainable=False)\nmodel.add(embedding_layer)\nmodel.add(Flatten())\nmodel.add(Dense(1, activation='sigmoid'))"

DEEP LEARNING 2

In [77]:
from keras.models import Input, Model
from keras.layers import Bidirectional, GlobalAvgPool1D, GlobalAveragePooling1D
from keras.layers import LSTM, Dense, Embedding, concatenate, Dropout, concatenate

inp1 = Input(shape=(length_long_sentence,))
x = Embedding(embedding_matrix.shape[0], 100, weights=[embedding_matrix])(inp1)
x = Bidirectional(LSTM(256, return_sequences=True))(x)
x = Bidirectional(LSTM(150))(x)
x = Dense(128, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(64, activation="relu")(x)
x = Dense(1, activation="sigmoid")(x)    
model = Model(inputs=inp1, outputs=x)

In [78]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
print(model.summary())

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 31)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 31, 100)           2269800   
_________________________________________________________________
bidirectional (Bidirectional (None, 31, 512)           731136    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 300)               795600    
_________________________________________________________________
dense (Dense)                (None, 128)               38528     
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)               

In [79]:
model.fit(padded_sentences_train, y_corpus, epochs=3, verbose=1)

Epoch 1/3
167/167 [==============================] - 79s 473ms/step - loss: 0.4674 - acc: 0.7855
Epoch 2/3
167/167 [==============================] - 79s 472ms/step - loss: 0.3710 - acc: 0.8465
Epoch 3/3
167/167 [==============================] - 81s 483ms/step - loss: 0.2800 - acc: 0.8880


In [80]:
prediction = model.predict(padded_sentences_test)

In [81]:
prediction = np.round(prediction)
prediction

array([[0.],
       [1.],
       [0.],
       ...,
       [0.],
       [0.],
       [1.]], dtype=float32)

In [84]:
prediction_final = pd.DataFrame({'id':test['id']})
prediction_final['target'] = prediction
prediction_final['target'] = prediction_final['target'].astype(int)
prediction_final.to_csv('prueba_glove_text_completo.csv', index=False)

In [85]:
prediction_final

,id,target
0,0,0
1,2,1
2,3,0
3,9,0
4,11,0
...,...,...
3258,10861,1
3259,10865,1
3260,10868,0
3261,10874,0
